<div align="center">
<h1><img width="30" src="https://madewithml.com/static/images/rounded_logo.png">&nbsp;<a href="https://madewithml.com/">Made With ML</a></h1>
Applied ML · MLOps · Production
<br>
Join 20K+ developers in learning how to responsibly <a href="https://madewithml.com/about/">deliver value</a> with applied ML.
</div>

<br>

<div align="center">
    <a target="_blank" href="https://madewithml.com/subscribe/"><img src="https://img.shields.io/badge/Subscribe-20K-brightgreen"></a>&nbsp;
    <a target="_blank" href="https://github.com/GokuMohandas/madewithml"><img src="https://img.shields.io/github/stars/GokuMohandas/madewithml.svg?style=social&label=Star"></a>&nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/goku"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>&nbsp;
    <a target="_blank" href="https://twitter.com/GokuMohandas"><img src="https://img.shields.io/twitter/follow/GokuMohandas.svg?label=Follow&style=social"></a>
    <p>🔥&nbsp; Among the <a href="https://github.com/topics/deep-learning" target="_blank">top ML</a> repositories on GitHub</p>
</div>

<br>
<hr>

# Optimize (GPU)

Use this notebooks to run hyperparameter optimization on Google Colab and utilize it's free GPUs.

In [ ]:
# In case you update the code after installing
%load_ext autoreload
%autoreload 2

## Clone repository

In [ ]:
# Load repository
!git clone https://github.com/GokuMohandas/applied-ml.git

In [ ]:
# Files
% cd applied-ml
!ls

## Setup

In [ ]:
# Use latest pip
!pip install --upgrade pip

In [ ]:
# Set up
!make install-dev

## Optimize

In [ ]:
from tagifai import main

In [ ]:
# Download data
main.download_data()

In [ ]:
# Check if data downloaded
!ls assets/data

In [ ]:
# Optimize
main.optimize(num_trials=50)

In [ ]:
# Train best model (saving artifacts this time)
main.train_model()

## Download

Download and transfer files to your local system and run the command `tagifai set-artifact-metadata` to match all metadata as if it were run from your machine.

In [ ]:
from google.colab import files

In [ ]:
# Download
!zip -r best.zip assets/experiments/1
files.download('best.zip') 
files.download('assets/experiments/trials.csv')
files.download('config/args.json') 